# Read SIFT 1 million 

In [287]:
using HDF5
using BenchmarkTools
using Distances
using LoopVectorization
using SIMD
using Clustering
using ProgressMeter
using StaticArrays
using DataFrames

In [288]:
Sys.cpu_info()[1].model

"Apple M1 Pro"

In [289]:
path = joinpath(homedir(), "Datasets", "SIFT1M",
                "sift-128-euclidean.hdf5")

"/Users/dbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5"

In [290]:
f = h5open(path, "r")

🗂️ HDF5.File: (read-only) /Users/dbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5
├─ 🏷️ distance
├─ 🔢 distances
├─ 🔢 neighbors
├─ 🔢 test
└─ 🔢 train

In [403]:
X_tr_vecs = read(f["train"])
X_te_vecs = read(f["test"]);
true_neighbors = read(f["neighbors"])
true_distances = read(f["distances"])

true_neighbors .= true_neighbors .+ 1;

@show size(X_tr_vecs)
@show size(X_te_vecs)
@show size(true_neighbors)
@show size(true_distances)

size(X_tr_vecs) = (128, 1000000)
size(X_te_vecs) = (128, 10000)
size(true_neighbors) = (100, 10000)
size(true_distances) = (100, 10000)


(100, 10000)

## Clustering.jl

Create the pqcodes for the training points

In [358]:
n_features = size(X_tr_vecs,1)
n_features

128

In [7]:
unique_values = sort(unique(X_tr_vecs))

213-element Vector{Float32}:
   0.0
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
  10.0
  11.0
  12.0
   ⋮
 202.0
 204.0
 205.0
 206.0
 207.0
 208.0
 210.0
 211.0
 212.0
 214.0
 216.0
 218.0

In [8]:
n_clusters = 16
R_per_feature = []
@showprogress for j in  1:n_features
    R = kmeans(X_tr_vecs[[j],:], n_clusters; maxiter=200)
    push!(R_per_feature, R)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:01:33


In [9]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[5.2500305, 67.759285, 119.13232, 39.83147, 30.818304, 129.33534, 93.13587, 22.87146, 11.353221, 16.917303, 79.71512, 140.89525, 107.12566, 0.49666122, 57.86853, 48.819954]
(16, 128)


In [10]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[5.2500305, 67.759285, 119.13232, 39.83147, 30.818304, 129.33534, 93.13587, 22.87146, 11.353221, 16.917303, 79.71512, 140.89525, 107.12566, 0.49666122, 57.86853, 48.819954]
(16, 128)


In [11]:
@benchmark P = hcat([r.centers[:] for r in R_per_feature]...)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.583 μs … 37.417 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.084 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.245 μs ±  1.029 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

             █▂    ▄ ▁▄                                        
  ▁▁▁▁▁▁▁▂▄▆███▇▇▅▇████▇▆▇▄▃▄▃▃▄▃▃▃▂▂▂▂▁▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  10.6 μs         Histogram: frequency by time        15.5 μs <

 Memory estimate: 28.47 KiB, allocs estimate: 137.

Another way to construct the matrix would be

In [12]:
function build_prototype_matrix(R, n_clusters, n_features)
    
    prototypes = Array{Float32}(undef, n_clusters, n_features);
    for j in 1:n_features
        prototypes[:,j] .= vec(R_per_feature[j].centers)
    end
    return prototypes
end

build_prototype_matrix (generic function with 1 method)

In [13]:
@benchmark P = build_prototype_matrix(R_per_feature, n_clusters, n_features)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  57.000 μs … 189.750 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     60.417 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   61.305 μs ±   4.919 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▄▆▇█▅▆▅▂▂▁                                               
  ▁▂▃▆████████████▆▆▆▄▅▃▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  57 μs           Histogram: frequency by time           76 μs <

 Memory estimate: 44.12 KiB, allocs estimate: 897.

In [489]:
P2 = build_prototype_matrix(R_per_feature, n_clusters, n_features);
isapprox(P, P2)

LoadError: DimensionMismatch: array could not be broadcast to match destination

We can generate the PQcodes for the data as follows:

In [488]:
PQcodes = hcat([Int32.(r.assignments[:]) for r in R_per_feature]...)';
size(PQcodes)

(128, 1000000)

Here each vector prototype is in fact a single scalar (vecause subvectors have a single coordinate)

In [18]:
size(P)

(16, 128)

We need a method to, given a vector and a collection of prototypes per feature, find which are the closest

In [19]:
query = X_te_vecs[:,1];

In [20]:
closest_prototypes = Array{UInt8}(undef, n_features, 1);

In [21]:
typeof(typemax(Float32))

Float32

In [22]:
@inline function seuclidean(x::T,y::T) where {T}
    aux::T = (x - y)
    return aux * aux
end

function find_closest_coordinate(dist, 
                                 value::T,
                                 vector) where T
    best_coordinate = 1
    min_distance::T = typemax(T)
    for (j,x) in enumerate(vector)
        current_dist = dist(x, value)
        if current_dist < min_distance
            best_coordinate = j
            min_distance = current_dist
        end
    end
    return best_coordinate
end

find_closest_coordinate (generic function with 1 method)

In [23]:
find_closest_coordinate(seuclidean, Float32(48.), query)
print(query)

Float32[1.0, 3.0, 11.0, 110.0, 62.0, 22.0, 4.0, 0.0, 43.0, 21.0, 22.0, 18.0, 6.0, 28.0, 64.0, 9.0, 11.0, 1.0, 0.0, 0.0, 1.0, 40.0, 101.0, 21.0, 20.0, 2.0, 4.0, 2.0, 2.0, 9.0, 18.0, 35.0, 1.0, 1.0, 7.0, 25.0, 108.0, 116.0, 63.0, 2.0, 0.0, 0.0, 11.0, 74.0, 40.0, 101.0, 116.0, 3.0, 33.0, 1.0, 1.0, 11.0, 14.0, 18.0, 116.0, 116.0, 68.0, 12.0, 5.0, 4.0, 2.0, 2.0, 9.0, 102.0, 17.0, 3.0, 10.0, 18.0, 8.0, 15.0, 67.0, 63.0, 15.0, 0.0, 14.0, 116.0, 80.0, 0.0, 2.0, 22.0, 96.0, 37.0, 28.0, 88.0, 43.0, 1.0, 4.0, 18.0, 116.0, 51.0, 5.0, 11.0, 32.0, 14.0, 8.0, 23.0, 44.0, 17.0, 12.0, 9.0, 0.0, 0.0, 19.0, 37.0, 85.0, 18.0, 16.0, 104.0, 22.0, 6.0, 2.0, 26.0, 12.0, 58.0, 67.0, 82.0, 25.0, 12.0, 2.0, 2.0, 25.0, 18.0, 8.0, 2.0, 19.0, 42.0, 48.0, 11.0]

In [24]:
@benchmark find_closest_coordinate(seuclidean, Float32(48.), query)


BenchmarkTools.Trial: 10000 samples with 780 evaluations.
 Range (min … max):  162.019 ns … 324.199 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     162.767 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   165.916 ns ±   6.713 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▃▃▂▂▃▆▄▂▂▅▃▃▂▂▂▂▁▂▁▁▁▁                                      ▂
  ██████████████████████████████▆▇█▇▇▇▆▆▆▆▆▆▆▇▆▇▆▅▆▆▅▆▇▅▅▆▅▅▄▅▄ █
  162 ns        Histogram: log(frequency) by time        191 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
@assert 3 == find_closest_coordinate(seuclidean, 2., [3.,5,2,1])

@assert 4 == find_closest_coordinate(seuclidean, 1., [3.,5,2,1])

In [26]:
function encode(dist, vector, prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    @inbounds for (j,x) in enumerate(vector)
            closest_prototypes[j] = find_closest_coordinate(dist, x, prototypes[:,j])
    end
    return closest_prototypes
end

encode (generic function with 1 method)

In [27]:
find_closest_coordinate(sqeuclidean, query[1], P[:,1])

14

In [28]:
encode(sqeuclidean, query, P)

128×1 Matrix{Int8}:
 14
 12
  1
  6
 14
 11
  2
  1
  2
 10
  4
 14
  3
  ⋮
 16
 11
  3
  2
 16
 10
  4
 11
 13
  5
  5
  6

In [29]:
@benchmark encode(sqeuclidean, query, P)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.048 μs …  10.952 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.845 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.890 μs ± 233.124 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                        ▅█▂▁▁▁▃▁                               
  ▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▅█████████▆▅▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  4.05 μs         Histogram: frequency by time        5.76 μs <

 Memory estimate: 16.19 KiB, allocs estimate: 129.

In [30]:
function encode_fast(dist, vector::Array{T}, prototypes::Array{T}) where T
    n_clusters, n_features = size(prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(prototypes[k,j],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
           #println(k, ' ', j, ' ', best_coordinate, ' ',min_distance )
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_fast (generic function with 1 method)

In [1]:
(query[1] .- P[:,1]) .* (query[1] .- P[:,1])

LoadError: UndefVarError: query not defined

In [32]:
println(find_closest_coordinate(sqeuclidean, query[1], P[:,1]))
println(find_closest_coordinate(sqeuclidean, query[2], P[:,2]))

14
12


In [33]:
encode_fast(sqeuclidean, query, P)[1:4]

4-element Vector{Int8}:
 14
 12
  1
  6

In [34]:
@assert isapprox(encode(sqeuclidean, query, P),
                 encode_fast(sqeuclidean, query, P))

In [35]:
@benchmark encode_fast(sqeuclidean, query, P)

BenchmarkTools.Trial: 10000 samples with 21 evaluations.
 Range (min … max):  956.333 ns …  3.452 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     980.190 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.002 μs ± 80.918 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅▄▂█▆▅▃▂▂▃▄▄▄▃▃▂▁                                            ▂
  ███████████████████▇▆▇▇▇▇▆▇▇▇█▇▆▇▆▆▆▆▄▅▅▄▃▄▃▄▅▁▁▄▃▄▅▃▃▄▁▅▃▄▅ █
  956 ns        Histogram: log(frequency) by time      1.32 μs <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [36]:
function compute_ADC(query, prototypes, dist)
    n_clusters, n_features = size(P)
    
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features
        for p in 1:n_clusters
            ADC_table[p,j] = dist(query[j], @view prototypes[p,:] )
        end
    end
    return  ADC_table
end

compute_ADC (generic function with 1 method)

In [37]:
size(P)

(16, 128)

In [38]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds   for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [39]:
@benchmark compute_ADC(query, P, SEuclidean0)

BenchmarkTools.Trial: 7457 samples with 1 evaluation.
 Range (min … max):  591.375 μs … 121.458 ms  ┊ GC (min … max): 0.00% … 99.43%
 Time  (median):     623.416 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   669.018 μs ±   1.458 ms  ┊ GC (mean ± σ):  2.42% ±  1.15%

     ▅█▁                                                         
  ▂▃▇███▅▅▆▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▁▂▁▂▂▁▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▁▂▂ ▃
  591 μs           Histogram: frequency by time         1.01 ms <

 Memory estimate: 272.25 KiB, allocs estimate: 10501.

In [40]:
adc_table = compute_ADC(query, P, SEuclidean0);

In [41]:
function compute_ADC_fast(query, prototypes, dist)
    n_clusters, n_features = size(P)
    
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features
        for p in 1:n_clusters

            ADC_table[p,j] = dist(query[j],  prototypes[p,:] )
        end
    end
    return  ADC_table
end

compute_ADC_fast (generic function with 1 method)

In [42]:
@benchmark compute_ADC_fast(query, P, SEuclidean0)

BenchmarkTools.Trial: 4799 samples with 1 evaluation.
 Range (min … max):  844.541 μs … 110.255 ms  ┊ GC (min … max):  0.00% … 99.12%
 Time  (median):     935.667 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.041 ms ±   3.334 ms  ┊ GC (mean ± σ):  10.33% ±  3.20%

                              ▁▂▁▂▃▂▄▄▆██▇▆▄▃▂                   
  ▃▄▃▃▃▃▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▂▂▃▄▅█▇████████████████▇▇▅▅▄▃▃▃▃▃▃▃▂▂▂▂▂ ▄
  845 μs           Histogram: frequency by time          994 μs <

 Memory estimate: 1.48 MiB, allocs estimate: 18693.

In [43]:
function adc_dist(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @fastmath for j in eachindex(x_code)
        res+= adc_table[x_code[j], j]
    end
    return res
end

adc_dist (generic function with 1 method)

In [44]:
query_code = encode_fast(sqeuclidean, query, P)
x_code = PQcodes[:,1];
adc_table = compute_ADC(query, P, SEuclidean0);
adc_dist(query_code, x_code,  adc_table)

552238.4f0

In [45]:
@benchmark adc_dist(query_code, x_code,  adc_table)

BenchmarkTools.Trial: 10000 samples with 972 evaluations.
 Range (min … max):  76.260 ns … 485.468 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     79.303 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   80.629 ns ±   8.051 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      █▁  ▇                                                     
  ▃▄▂███▃▅█▇▇▆▄▃▂▃▃▄▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  76.3 ns         Histogram: frequency by time         97.4 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [46]:
x = X_tr_vecs[:,1];
@benchmark SEuclidean0(x, query)

BenchmarkTools.Trial: 10000 samples with 925 evaluations.
 Range (min … max):  111.531 ns … 493.963 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     115.045 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   115.763 ns ±   6.349 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

           █▄        ▅                                           
  ▂▂▂▂▂▅▄▃▆███▄▃▄▃▃▄██▆▅▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂ ▃
  112 ns           Histogram: frequency by time          126 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

##  Share prototypes across features

https://groups.google.com/g/julia-users/c/xBcQRebyi_o



In [47]:
n_clusters = 32
#R_shared = kmeans(Matrix(vec(X_tr_vecs)'), n_clusters; maxiter=200)
R_shared = kmeans(X_tr_vecs[[1],:], n_clusters; maxiter=200)

KmeansResult{Matrix{Float32}, Float32, Int64}(Float32[10.896834 47.46942 … 117.01372 71.43038], [14, 21, 14, 1, 10, 2, 14, 21, 9, 30  …  27, 1, 1, 8, 28, 11, 29, 8, 14, 18], Float32[0.0, 2.078003, 0.0, 1.21698, 0.19084167, 0.28125, 0.0, 0.31188965, 1.0723343, 2.3806763  …  0.00024414062, 0.80430603, 3.5979767, 2.3684082, 0.21459961, 0.9169922, 0.8599529, 2.1346436, 0.0, 0.81640625], [83245, 21109, 9623, 39983, 13862, 5061, 11962, 31501, 63822, 82460  …  14845, 842, 18874, 13205, 19110, 28233, 82724, 45840, 6925, 18207], [83245, 21109, 9623, 39983, 13862, 5061, 11962, 31501, 63822, 82460  …  14845, 842, 18874, 13205, 19110, 28233, 82724, 45840, 6925, 18207], 1.281937f6, 4, true)

In [48]:
function encode_shared(dist, vector::Array{T}, shared_prototypes::Array{T}) where T
    n_clusters = length(shared_prototypes)
    
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(shared_prototypes[k,j],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
           #println(k, ' ', j, ' ', best_coordinate, ' ',min_distance )
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_shared (generic function with 1 method)

In [49]:
maximum(R_shared.assignments)

32

In [92]:
P_shared = R_shared.centers

1×32 Matrix{Float32}:
 10.8968  47.4694  120.508  23.4558  …  3.92734  19.4571  117.014  71.4304

In [93]:
j=3
encode_shared(sqeuclidean, X_tr_vecs[:,j] ,P_shared) 

128×1 Matrix{Int8}:
 14
  9
 27
 26
 15
 22
 24
 14
  1
  4
  6
 18
  7
  ⋮
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1

In [94]:
size(x), size(X_tr_vecs[:,1])

((128,), (128,))

In [490]:
PQcodes_shared = Array{Int16}(undef, n_features, n_examples);

for j in 1:n_examples
    PQcodes_shared[:,j] = encode_shared(sqeuclidean, X_tr_vecs[:,j] ,P_shared)  
end

In [96]:
maximum(PQcodes), maximum(PQcodes_shared)

(16, 32)

Now we need a new encoding function that uses a single vector of prototypes

In [97]:
function encode_fast_shared(dist, vector::Array{T}, prototypes::Array{T}) where T
    
    n_features = length(vector)
    n_clusters = length(prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds @fastmath for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(prototypes[k],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_fast_shared (generic function with 1 method)

In [98]:
@benchmark encode_fast_shared(sqeuclidean, query, P_shared)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.729 μs …   6.404 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.792 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.809 μs ± 130.347 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▄▅▂   ▂▆▅▃▃▂ ▂▁▄▄▃▃▃▂▂▂ ▂▁ ▁ ▁                            ▂
  █████▇████████████████████████▇██▇██▇█▇▆▇▇▅▇▆▇▆▅▄▅▄▅▅▄▄▄▄▄▄ █
  1.73 μs      Histogram: log(frequency) by time      2.13 μs <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [521]:
function compute_ADC_shared(query, prototypes, dist)
    n_clusters = length(prototypes)
    ADC_table = Array{Float32}(undef, n_clusters)
    
    @inbounds  for p in 1:n_clusters            
        ADC_table[p] = dist(query[p],  prototypes[p] )
    end
    return  ADC_table
end

query_code = encode_fast_shared(sqeuclidean, query, P_shared)
x_code = PQcodes[:,1];
adc_table_shared = Float32.(compute_ADC_shared(query, P_shared, SEuclidean0));
#adc_dist(query_code, x_code,  adc_table)

In [101]:
@benchmark adc_table = compute_ADC_shared(query, P_shared, SEuclidean0)

BenchmarkTools.Trial: 10000 samples with 979 evaluations.
 Range (min … max):  63.925 ns … 117.749 μs  ┊ GC (min … max):  0.00% … 99.90%
 Time  (median):     77.992 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   95.438 ns ±   1.178 μs  ┊ GC (mean ± σ):  12.33% ±  1.00%

           ▃▇█▄▄▂                                               
  ▂▁▃▁▁▂▁▂▅███████▇▆▆▄▄▃▃▃▃▄▅▅▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  63.9 ns         Histogram: frequency by time          116 ns <

 Memory estimate: 192 bytes, allocs estimate: 1.

## Improving distance between vectors with ADC_distance

In [102]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds   for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [103]:
@benchmark SEuclidean0($x, $query)

BenchmarkTools.Trial: 10000 samples with 968 evaluations.
 Range (min … max):  77.436 ns … 153.022 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     77.651 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   79.118 ns ±   3.061 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▃▁▁▁▁▁▁▁▆▃▃▂▁▁▁▁▁▁▂▁                                        ▁
  ███████████████████████▇▇▇▇▆▆▆▅▅▃▄▄▄▄▅▃▃▃▃▃▃▄▄▃▅▄▃▄▅▅▅▄▃▃▂▃▄ █
  77.4 ns       Histogram: log(frequency) by time        93 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [104]:
function adc_dist_shared(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(x_code)
        res+= adc_table[x_code[j]]
    end
    return res
end

adc_dist_shared (generic function with 1 method)

In [105]:
@benchmark adc_dist_shared($query_code, $x_code, $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  38.810 ns … 91.272 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.252 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.698 ns ±  1.491 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▆▆█▅▄▃▂    ▁▂▄▄▄▂▁         ▁   ▁                          ▂
  █████████▇▇▇██████████▆█▇▇▇▇▇██████▇▆▆▅▆▅▄▅▃▅▅▅▅▄▄▃▄▄▂▄▄▄▅▅ █
  38.8 ns      Histogram: log(frequency) by time      44.4 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [538]:
function adc_dist_shared_unrolled(query_code, x_code,  adc_table)
    res1 = zero(eltype(adc_table))
    res2 = zero(eltype(adc_table))
    res3 = zero(eltype(adc_table))
    res4 = zero(eltype(adc_table))
    
    @inbounds @fastmath for j in 1:4:length(x_code)
        res1 += adc_table[x_code[j]]
        res2 += adc_table[x_code[j+1]]
        res3 += adc_table[x_code[j+2]]
        res4 += adc_table[x_code[j+3]]
    end
    
    return res1 + res2 + res3 + res4
end

adc_dist_shared_unrolled (generic function with 1 method)

In [540]:
@benchmark adc_dist_shared_unrolled($query_code, $x_code,  $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 995 evaluations.
 Range (min … max):  28.727 ns … 114.865 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     28.894 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.502 ns ±   2.371 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅ ▁▅▄▁ ▁▂▂▁  ▁                                              ▁
  ████████████▇██▆▆▆▇▇▆▆▆▇▇▆▆▆▅▁▄▅▆▅▅▅▆▅▅▅▅▅▄▅▄▃▃▄▁▃▁▃▁▃▄▃▅▄▅▅ █
  28.7 ns       Histogram: log(frequency) by time        41 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [541]:
@benchmark adc_dist_shared($query_code, $x_code,  $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  38.894 ns … 143.608 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.273 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.727 ns ±   2.349 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▇▃▁  ▂▃       ▁                                            ▁
  ██████████▇▇▆▇███▆▆▅▅▄▅▅▄▄▆▅▅▅▅▃▃▃▃▃▅▃▄▄▄▄▄▃▄▂▂▄▃▃▂▃▃▂▃▄▂▃▄▄ █
  38.9 ns       Histogram: log(frequency) by time      50.7 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [529]:
adc_dist_shared(query_code, x_code,  adc_table_shared)

579822.0f0

In [532]:
adc_dist(query_code, x_code,  adc_table)

552238.4f0

In [533]:
adc_dist_shared_unrolled(query_code, x_code,  adc_table_shared)

579821.9f0

Note that euclidean squared distance with `@fastmath` is still faster

In [110]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds @fastmath  for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [111]:
@benchmark SEuclidean0($x, $query)

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  9.551 ns … 28.444 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.635 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.701 ns ±  0.466 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁ ▆  █ ▄  ▂ ▃  ▁      ▃                 ▁ ▁                ▁
  █▁█▁▁█▁█▁▁█▁█▁▁█▁▅▁▁█▁█▁▁█▁█▁▁█▁▇▁▁█▁█▁▁█▁█▁▁▇▁▇▁▁▇▁▇▁▁▆▁▆ █
  9.55 ns      Histogram: log(frequency) by time     10.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

What if we use an adctable cotaining only integers?

In [112]:
function adc_dist_shared(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(x_code)
        res+= adc_table[x_code[j]]
    end
    return res
end

adc_dist_shared (generic function with 1 method)

In [113]:
adc_table_shared_int = Int32.(round.(adc_table_shared));

In [114]:
@benchmark adc_dist_shared($query_code, $x_code,  $adc_table_shared_int)

BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  39.315 ns … 135.459 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.650 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   40.034 ns ±   1.619 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▅██▇▅▃▁    ▁  ▁▃▃▂▁            ▁                            ▂
  ████████▇▇▇▇█████████▇▇▇▇▆▇▇▇▇███████▇▇▇▇▆▅▆▆▆▅▅▅▆▅▄▅▅▄▄▄▄▂▃ █
  39.3 ns       Histogram: log(frequency) by time      44.6 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

## BWcode

When using a SharedQuantizer a value `c` in a pqcode found in different coordinates refers to the same real number. Note that the computation of an ADC distance  is a sum over  over ADC_table entries provided by the pqcode. This sum is done over the entire length of the pqcode. 

We propose to create a  `K`-dimensional vector, that we call BWcode, that compresses a pqcode into a vector containing the counts of the different assigments to each centroid prototype over the pqcode. For example if we had a pqcode `[1,2,4,1,1,3,3,4]` represented with 4 centroids, we would create a bw code `[3,1,2,2]`. Then the adcomputation over the pqcode is the same as the adc computation over a bwcode provided that we multiply each adc_table entry by the value in the bwcode.

To sum up, we can iterate over a `K`-dimensional vector that we call the BWcode and add the values in `adc_table_shared` weighted by the amount of times each prototype has been seen in the pqcode, making the computation faster as long as `K` is lower than the length of the pqcode.

In [480]:
function pq_code_to_bw_code(pqcode, n_clusters)
    bw_code = zeros(eltype(pqcode), n_clusters)
    for c in pqcode
        bw_code[c] += 1
    end
    return bw_code
end

pq_code_to_bw_code (generic function with 1 method)

In [360]:
n_examples = size(X_tr_vecs,2)
BWcodes = Array{Int16}(undef, n_clusters, n_examples);
bw_code = pq_code_to_bw_code(x_code, n_clusters);

Now let´s verify that the distance between a query

In [361]:
function bw_adc_dist_shared(query_code, bw_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(bw_code)
        res+= adc_table[j] * bw_code[j]
    end
    return res
end

bw_adc_dist_shared (generic function with 1 method)

In [118]:
@benchmark bw_adc_dist_shared($query_code, $bw_code, $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  4.583 ns … 15.042 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.667 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.682 ns ±  0.199 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▃         █        ▄▃        ▂         ▂          ▁
  ▅▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█ █
  4.58 ns      Histogram: log(frequency) by time     4.83 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [482]:
@assert isapprox(bw_adc_dist_shared(query_code, bw_code, adc_table_shared),
                 adc_dist_shared(query_code, x_code, adc_table_shared))

## Linear scann study

### Exact version

In [121]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds @fastmath  for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

function linear_scann_exact(query, X)

    n_features, n_examples = size(X)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath  for j in 1:n_examples
        distances[j] = SEuclidean0(query, X[:,j])    
    end
    return distances
end

@benchmark linear_scann_exact(query, X_tr_vecs)

BenchmarkTools.Trial: 27 samples with 1 evaluation.
 Range (min … max):  137.661 ms … 347.214 ms  ┊ GC (min … max):  0.00% … 38.55%
 Time  (median):     148.668 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   194.592 ms ±  68.133 ms  ┊ GC (mean ± σ):  24.97% ± 23.30%

  █▃▁                                                            
  ███▄▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▄▁▇▄▁▄▄▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  138 ms           Histogram: frequency by time          347 ms <

 Memory estimate: 553.13 MiB, allocs estimate: 1000002.

In [122]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

function linear_scann_exact(query, X)

    n_features, n_examples = size(X)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath Threads.@threads for j in 1:n_examples
        distances[j] = SEuclidean0(query, X[:,j])    
    end
    return distances
end

@benchmark linear_scann_exact(query, X_tr_vecs)

BenchmarkTools.Trial: 23 samples with 1 evaluation.
 Range (min … max):   45.934 ms … 394.457 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     242.742 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   217.840 ms ±  81.662 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄                           ▁▁    █▁ ▁▁                        
  █▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆██▁▁▆▁██▆██▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  45.9 ms          Histogram: frequency by time          394 ms <

 Memory estimate: 553.14 MiB, allocs estimate: 1000064.

In [123]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds @fastmath  for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

function linear_scann_exact(query, X)

    n_features, n_examples = size(X)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = SEuclidean0(query, view(X,:,j))    
    end
    return distances
end

@benchmark linear_scann_exact(query, X_tr_vecs)

BenchmarkTools.Trial: 478 samples with 1 evaluation.
 Range (min … max):   9.932 ms … 41.107 ms  ┊ GC (min … max): 0.00% … 73.98%
 Time  (median):     10.398 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.451 ms ±  1.419 ms  ┊ GC (mean ± σ):  0.61% ±  3.38%

              ▇█▅▆         ▅▅▃▄▂                               
  ▂▁▁▁▁▁▁▁▁▁▁██████▇▇▅▅▅▅▄▆██████▇▇▆▅▄▃▃▂▃▂▂▃▂▄▃▃▁▂▂▂▃▂▄▂▁▃▂▂ ▄
  9.93 ms         Histogram: frequency by time          11 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

### ADC version

In [124]:
function linear_scann(query, PQcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(PQcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath for j in 1:n_examples
        distances[j] = adc_dist_shared(query_code,  view(PQcodes,:,j) ,  adc_table_shared)    
    end
    return distances
end

@benchmark distances = linear_scann($query, $PQcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 56 samples with 1 evaluation.
 Range (min … max):  88.517 ms …  90.538 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     90.193 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   90.110 ms ± 366.821 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                                  ▄ ▂█▄▂        
  ▄▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▄▁▁▁▁▄▄▁▁▁▆▁▄▁▁██▆████▆▆▆███ ▁
  88.5 ms         Histogram: frequency by time         90.5 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 3.

In [126]:
function linear_scann_unrolled(query, PQcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(PQcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    
    @inbounds @fastmath Threads.@threads for j in 1:n_examples
        distances[j] = adc_dist_shared_unrolled(query_code,  PQcodes[:,j] ,  adc_table_shared)    
    end
    return distances
end

@benchmark distances = linear_scann_unrolled($query, $PQcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 65 samples with 1 evaluation.
 Range (min … max):  40.275 ms …    1.163 s  ┊ GC (min … max):  0.00% … 95.32%
 Time  (median):     43.677 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   77.351 ms ± 193.407 ms  ┊ GC (mean ± σ):  43.82% ± 16.61%

  █                                                             
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  40.3 ms       Histogram: log(frequency) by time       1.15 s <

 Memory estimate: 553.14 MiB, allocs estimate: 1000064.

### ADC version with BW codes

Now we will build a Matrix containing BW codes and use it to compute the ADC distance

In [483]:
n_examples = size(X_tr_vecs,2)
BWcodes = Array{Int8}(undef, n_clusters, n_examples);
bw_code = pq_code_to_bw_code(x_code, n_clusters);

for j in 1:n_examples
   BWcodes[:,j] .= pq_code_to_bw_code(PQcodes_shared[:,j], n_clusters)
end

In [484]:
function bw_adc_dist_shared(query_code, bw_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(bw_code)
        res+= adc_table[j] * bw_code[j]
    end
    return res
end

function linear_scann_bw(query, BWcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(BWcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    n_clusters = size(BWcodes,1)
    #@show n_clusters
    @inbounds for i in 1:n_examples
        
        # Following code is equivalent to bw_adc_dist_shared(query_code, BWcodes[:,i],  adc_table_shared)
        res = zero(eltype(adc_table_shared))
        @simd for j in 1:n_clusters
            res+= adc_table_shared[j] * BWcodes[j,i]
        end        
        distances[i] = res  
    
    end
    return distances
end

@benchmark linear_scann_bw($query, $BWcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 1468 samples with 1 evaluation.
 Range (min … max):  3.298 ms …   7.919 ms  ┊ GC (min … max): 0.00% … 51.67%
 Time  (median):     3.318 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.400 ms ± 239.430 μs  ┊ GC (mean ± σ):  1.63% ±  4.58%

  █▆▄▄▄▃▂▁                                           ▁         
  ████████▇▆▇▇▄▅▄▄▄▆▆▄▄▄▅▄▄▄▁▄▄▄▆▅▄▄▄▄▅▁▄▄▄▄▄▄▄▄▄▇▇▆▆███▇▇▇▆▆ █
  3.3 ms       Histogram: log(frequency) by time      4.06 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 3.

In [485]:
function bw_adc_dist_shared(query_code, bw_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(bw_code)
        res+= adc_table[j] * bw_code[j]
    end
    return res
end

function linear_scann_bw_multithreaded(query, BWcodes, adc_table_shared, P_shared)
    
    query_code = encode_fast_shared(sqeuclidean, query, P_shared)
    n_features, n_examples = size(BWcodes)
    distances = Array{eltype(query)}(undef, n_examples)
    n_clusters = size(BWcodes,1)
    #@show n_clusters
    @inbounds @fastmath Threads.@threads  for i in 1:n_examples
       distances[i] = bw_adc_dist_shared(query_code, view(BWcodes,:,i),  adc_table_shared)
    end    
    
    return distances
end

@benchmark distances = linear_scann_bw_multithreaded($query, $BWcodes, $adc_table_shared, $P_shared)

BenchmarkTools.Trial: 5604 samples with 1 evaluation.
 Range (min … max):  702.541 μs …   6.133 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     806.188 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   884.525 μs ± 299.577 μs  ┊ GC (mean ± σ):  8.90% ± 14.47%

  ▂██▇█▇▅▅▄▂▁                  ▁  ▁▁                            ▂
  ███████████▇▇▇▇▅▆▅▄▃▃▅▃▁▃▁▁▃▇██████▇█▇▇▇▇▆▇▆▇▇▅▇▆▆▅▄▅▆▆▆▆██▇▇ █
  703 μs        Histogram: log(frequency) by time       2.19 ms <

 Memory estimate: 3.82 MiB, allocs estimate: 64.

# Quality verification

Let's inspect the recall of pqlinearscann with respect to exact search:

In [486]:
    
function recall(predicted, relevant, eval_at)
    """
    fraction of the relevant documents that are successfully retrieved
    """
    if eval_at == 0
        return 0.0
    end
    
    predicted_at_k = predicted[1:eval_at]
    n_predicted_and_relevant =  length(intersect( Set(predicted_at_k), Set(relevant))) 
    return n_predicted_and_relevant/ length(relevant)
end

recall (generic function with 1 method)

In [510]:
query_id = 1
query = X_te_vecs[:,query_id];
query_true_neighbors = neighbors[:,query_id]
top_k = 100

bw_distances = linear_scann_bw(query, BWcodes, adc_table_shared, P_shared);
top_k_bw = sortperm(distances)[1:top_k];

pq_distances = linear_scann(query, PQcodes, adc_table_shared, P_shared);
top_k_pq = sortperm(distances)[1:top_k];

ex_distances = linear_scann_exact(query, X_tr_vecs)
top_k_ex = sortperm(ex_distances)[1:top_k];

@show precision(top_k_ex, query_true_neighbors, 100)
@show precision(top_k_pq, query_true_neighbors, 100)
@show precision(top_k_bw, query_true_neighbors, 100)

df = DataFrame(Dict(:top_k_ex => top_k_ex,
                    :top_k_pq => top_k_pq,
                    :top_k_bw => top_k_bw))

precision(top_k_ex, query_true_neighbors, 100) = 1.0
precision(top_k_pq, query_true_neighbors, 100) = 0.0
precision(top_k_bw, query_true_neighbors, 100) = 0.0


,top_k_bw,top_k_ex,top_k_pq
,Int64,Int64,Int64
1,237541,932086,237541
2,257186,934877,257186
3,299118,561814,299118
4,741233,708178,741233
5,884572,706772,884572
6,806278,695757,806278
7,701985,435346,701985
8,195172,701259,195172
9,273988,455538,273988


In [519]:
using Plots
x = 1:10; y = rand(10); # These are the plotting data
plot(x, y)

LoadError: UndefVarError: plot not defined

In [513]:
X_tr_vecs[:,1]

128-element Vector{Float32}:
  0.0
 16.0
 35.0
  5.0
 32.0
 31.0
 14.0
 10.0
 11.0
 78.0
 55.0
 10.0
 45.0
  ⋮
  9.0
 17.0
 23.0
 41.0
  0.0
  0.0
  2.0
  8.0
 19.0
 25.0
 23.0
  1.0

In [400]:
n = 3
sqeuclidean(query, X_tr_vecs[:,n]) , distances[n], sqeuclidean(sqeuclidean(query, X_tr_vecs[:,n]), distances[n])

(410871.0f0, 536488.94f0, 1.5779867f10)

In [401]:
n = 4
sqeuclidean(distances[n], sqeuclidean(sqeuclidean(query, X_tr_vecs[:,n]), distances[n])

(416693.0f0, 426236.4f0, 9.10766f7)

In [402]:
n = 1
sqeuclidean(query, X_tr_vecs[:,n]) , distances[n], sqeuclidean(sqeuclidean(query, X_tr_vecs[:,n]), distances[n])

(378949.0f0, 579822.0f0, 4.0349962f10)

## Euclidean  Squared Distance

In [440]:
function SEuclidean(X, query)
    d = (query .- X) .* (query .- X)
    return sum(d, dims=1)
end

SEuclidean (generic function with 1 method)

In [441]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

SEuclidean_2 (generic function with 1 method)

In [442]:
query = X_te_vecs[:,1];

In [443]:
@benchmark SEuclidean(X_te_vecs, query)

BenchmarkTools.Trial: 3753 samples with 1 evaluation.
 Range (min … max):  928.708 μs … 32.129 ms  ┊ GC (min … max): 0.00% … 96.23%
 Time  (median):       1.211 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.328 ms ±  1.582 ms  ┊ GC (mean ± σ):  7.66% ±  6.17%

                      ▅▇██▇▆▆▅▅▅▄▂     ▂▃▃▂▂▁▁                  
  ▅▅▄▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▄▇██████████████▇▇████████▅▅▅▄▄▃▃▃▃▃▃▂▃▂▂▂ ▅
  929 μs          Histogram: frequency by time          1.5 ms <

 Memory estimate: 4.92 MiB, allocs estimate: 4.

In [444]:
@benchmark SEuclidean_2(X_te_vecs, query)

BenchmarkTools.Trial: 6369 samples with 1 evaluation.
 Range (min … max):  769.583 μs …  2.304 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     780.625 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   783.792 μs ± 22.694 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▂▂▄▅▆█▆▇▆▃▁                                               
  ▃▃▃████████████▇▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  770 μs          Histogram: frequency by time          831 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [445]:
@benchmark SEuclidean(X_tr_vecs, query)

BenchmarkTools.Trial: 39 samples with 1 evaluation.
 Range (min … max):  115.288 ms … 213.858 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     121.950 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   131.411 ms ±  21.901 ms  ┊ GC (mean ± σ):  5.18% ± 6.91%

  ▄█           ▁                                                 
  ██▇▆▄▆▄▄▁▁▁▁▄█▇▆▁▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄ ▁
  115 ms           Histogram: frequency by time          214 ms <

 Memory estimate: 492.10 MiB, allocs estimate: 4.

In [446]:
@benchmark SEuclidean_2(X_tr_vecs, query)

BenchmarkTools.Trial: 65 samples with 1 evaluation.
 Range (min … max):  77.253 ms …  79.056 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     77.805 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   77.872 ms ± 390.832 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▃▆█ ▆         █    ▁                                 
  ▇▁▁▁▁▁▄▁▇███▇█▄▄▁▇▄▇▄▇▁█▇▄▇▁█▄▁▁▁▄▁▁▄▁▇▄▄▄▄▁▄▁▁▁▄▁▁▁▁▁▄▁▁▁▁▄ ▁
  77.3 ms         Histogram: frequency by time         78.9 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

## Finding top k distances (and their ids)


The first naive thing we can do consist on computing all distances and then sorting them to get the top k closest vectors to the query vector

In [447]:
function top_k_ids(X, query)
    distances = SEuclidean_2(X, query)
    top_k_indices = sortperm(distances)
    return top_k_indices
end

top_k_ids (generic function with 1 method)

In [448]:
@benchmark top_k_ids(X_te_vecs, query)[1:10]

BenchmarkTools.Trial: 3988 samples with 1 evaluation.
 Range (min … max):  1.228 ms …  1.796 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.247 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.252 ms ± 20.688 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁▅▅▇█▅▆▃▂                                              
  ▂▃▃▅▆███████████▇▅▅▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.23 ms        Histogram: frequency by time        1.32 ms <

 Memory estimate: 156.50 KiB, allocs estimate: 6.

A slightly better approach consist on using `partialsortperm` to simply sort a subset of the distances vector.

In [449]:
function top_k_ids_2(X, query, k)
    distances = SEuclidean_2(X, query)
    top_k_indices = partialsortperm(distances, 1:k)
    return top_k_indices
end

top_k_ids_2 (generic function with 1 method)

In [450]:
@benchmark top_k_ids_2(X_te_vecs, query, 10)

BenchmarkTools.Trial: 6056 samples with 1 evaluation.
 Range (min … max):  785.709 μs …  1.346 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     808.042 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   822.902 μs ± 37.672 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁▇█▄▁                                                      
  ▃▃▃█████▇▆▅▄▄▃▃▄▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  786 μs          Histogram: frequency by time          944 μs <

 Memory estimate: 156.44 KiB, allocs estimate: 7.

### Storing top k distances in a priority queue

A better alternative consists on using a priority queue. This. queue will keep only k distances in memory (no need to store all distances between the query point and all possible candidates).

In [451]:
sort!([1,54,3,24,10])

5-element Vector{Int64}:
  1
  3
 10
 24
 54

In [452]:
a = [1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [453]:
function SEuclidean_3(X, query, top_k)
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res/n_features
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end
            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end

    end
    return result
end

SEuclidean_3 (generic function with 1 method)

In [454]:
@benchmark SEuclidean_3(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 62 samples with 1 evaluation.
 Range (min … max):  79.984 ms … 90.506 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     80.591 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   81.169 ms ±  1.771 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

       █▂                                                      
  ▆▇▇▆▇██▁▅▁▄▁▃▁▃▃▁▃▃▁▃▃▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▃▁▁▁▁▁▁▁▃▁▁▁▃ ▁
  80 ms           Histogram: frequency by time        86.2 ms <

 Memory estimate: 16.67 KiB, allocs estimate: 120.

In [455]:
@benchmark top_k_ids_2(X_tr_vecs, query, 10)

BenchmarkTools.Trial: 54 samples with 1 evaluation.
 Range (min … max):  86.799 ms … 150.780 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     89.566 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   93.604 ms ±  11.673 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃ ▃█                                                         
  ▄████▇▄▁▃▁▁▁▄▄▁▁▁▁▁▃▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▁▁▁▃ ▁
  86.8 ms         Histogram: frequency by time          125 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 7.

In [456]:
function SEuclidean_4(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    @inbounds @fastmath for m in top_k:n_examples
        res = zero(eltype(X))
        @simd for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end


SEuclidean_4 (generic function with 1 method)

In [462]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 416 samples with 1 evaluation.
 Range (min … max):  11.872 ms …  13.198 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.996 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.029 ms ± 161.525 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█              ▁  ▁                                          
  ███▆▅▆▅▄▃▅▃▄▃▃▃▃██▇█▅▄▂▃▃▃▂▃▁▂▁▁▁▂▁▂▃▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂ ▃
  11.9 ms         Histogram: frequency by time         12.7 ms <

 Memory estimate: 15.97 KiB, allocs estimate: 114.

In [458]:
X_tr_200k = X_tr_vecs[:,1:200_000]
@benchmark SEuclidean_4(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 2022 samples with 1 evaluation.
 Range (min … max):  2.424 ms …  3.359 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.448 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.463 ms ± 52.802 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▅▄▁                                                       
  ██████▇▇▆▆▅▆▆▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▂▁▁▂▂ ▃
  2.42 ms        Histogram: frequency by time        2.68 ms <

 Memory estimate: 13.94 KiB, allocs estimate: 94.

In [ ]:
SEuclidean_4(X_tr_vecs, query, 10)

In [459]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        @turbo  for j in 1:n_features
            aux = (query[j] - X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

SEuclidean_5 (generic function with 1 method)

In [460]:
@benchmark SEuclidean_5(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 2251 samples with 1 evaluation.
 Range (min … max):  2.148 ms …  2.755 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.197 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.211 ms ± 58.715 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▄▇██▇▆▆▇▇▆▅▄▃▂▂▁▁                                        ▂
  ▄▄██████████████████▆█▆▆▇▇▆▇▇▆█▇▆▇▅▅▅▆▄▁▄▅▁▄▁▅▄▄▅▁▄▅▁▅▅▅▁▆ █
  2.15 ms      Histogram: log(frequency) by time     2.52 ms <

 Memory estimate: 13.94 KiB, allocs estimate: 94.

# Comparison with Distances.jl


Distances.jl provides the method 'pairwise' to compute pairwise distances between two matrices.

In [ ]:
@show typeof(query)
@show size(query)
@show typeof(X_tr_vecs)
@show size(X_tr_vecs)

In [ ]:
v1 = [1,2,3]
v2 = [0,0,1]

y = [0,0,2]
X = [v1 v2]

In [ ]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_5(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        dist = zero(eltype(X))
        @turbo  for j in 1:n_features
            dist = (query[j] - X[j,m])
            dist += aux * aux
        end
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

In [ ]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(eltype(X), n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

In [ ]:
y_colvec = [y y][:,[1]]
query_mat = [query query][:,[1]];

In [ ]:
SEuclidean_2(X, y)

In [ ]:
pairwise(SqEuclidean(), y_colvec, X)

In [ ]:
@benchmark pairwise(SqEuclidean(), query_mat, X_tr_vecs)

In [ ]:
@benchmark SEuclidean_2(X_tr_vecs, query)

In [ ]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10)

We can verify that the sorted distances by `SEuclidean_4` are the same as the results provided with `pairwise` from `Distances.jl`

In [ ]:
res2 = SEuclidean_4(X_tr_vecs, query, 10);
res = pairwise(SqEuclidean(), query_mat, X_tr_vecs)
res = sort(res, dims=2)[1:10]
res == res2

## Parallel implementation

# SIMD tests

In [ ]:

function find_val_in_array_simd(x::Array{T}, val::T) where {T}
    n_simd = 64
    last_pos_simd_chunk = length(x)-n_simd
    @inbounds for i in 1:n_simd:last_pos_simd_chunk
        vec_i = vload(Vec{n_simd, T}, x, i)
        sum_equality = sum(vec_i == val)
        if sum_equality >0
            return true
        end
    end

    @inbounds for i in last_pos_simd_chunk:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end

In [ ]:
function jdotavx(a, b)
    s = zero(eltype(a))
    @turbo for i ∈ eachindex(a, b)
        s += a[i] * b[i]
    end
    s
end